In [ ]:
import os, sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
Aggregate = pd.read_excel('aggregate_20010101-20220314_annotated_4-11-22 (2).xlsx',header=1)
Notes = pd.read_excel('CTP_ClarityNotes_N=151_v3.xlsx',header=0)

In [ ]:
MRNs = pd.read_csv('Included MRNs (120)',header=0,index_col=0)['PatientID'].map(str)
MRNs


In [ ]:
Aggregate['PatientID'] = Aggregate['PatientID'].map(str)
Aggregate_Included = Aggregate[Aggregate['PatientID'].isin(MRNs)]
Aggregate_Included_CTP = Aggregate_Included[Aggregate_Included['StudyDescription'].str.contains('P', case=False, na=False)]

Aggregate_Included_CTP_WithRatio = Aggregate_Included_CTP[pd.notna(Aggregate_Included_CTP['P:C(30)>=1.8'])]
Aggregate_Included_CTP_WithRatio_AndPenumbra = Aggregate_Included_CTP_WithRatio[Aggregate_Included_CTP_WithRatio['>6s']!= 0]
Included_Subjects_Aggregate = Aggregate_Included_CTP_WithRatio_AndPenumbra.sort_values(['StudyDate','StudyTime']).drop_duplicates('PatientID').sort_index()

Included_Subjects_Aggregate

In [ ]:
Aggregate_Included_WithRatio = Aggregate_Included[pd.notna(Aggregate_Included['P:C(30)>=1.8'])]
Aggregate_Included_WithRatio_AndPenumbra = Aggregate_Included_WithRatio[Aggregate_Included_WithRatio['>6s']!= 0]
Included_Subjects_FirstScanTimes = Aggregate_Included_WithRatio_AndPenumbra.sort_values(['StudyDate','StudyTime']).drop_duplicates('PatientID').sort_index()[['PatientID','StudyDate','StudyTime','StudyDescription']]
Included_Subjects_FirstScanTimes

In [ ]:
def pad(lkwTime):
    x = str(int(lkwTime))
    while len(x)<6:
        x = '0' + x
    return(x)

In [ ]:
Included_Subjects_Aggregate['ScanTime'] = Included_Subjects_Aggregate['StudyDate'].apply(pad) + Included_Subjects_Aggregate['StudyTime'].apply(pad)
Included_Subjects_Aggregate['ScanTime'] = pd.to_datetime(Included_Subjects_Aggregate['ScanTime'],format='%Y%m%d%H%M%S') 
Included_Subjects_Aggregate['LKWTime'] = Included_Subjects_Aggregate['LKW date'].apply(pad) + Included_Subjects_Aggregate['LKW time'].apply(pad)
Included_Subjects_Aggregate['LKWTime'] = pd.to_datetime(Included_Subjects_Aggregate['LKWTime'],format='%Y%m%d%H%M%S') 
Included_Subjects_Aggregate['LKWDiff'] = Included_Subjects_Aggregate['ScanTime'] - Included_Subjects_Aggregate['LKWTime']
Included_Subjects_Aggregate = Included_Subjects_Aggregate.drop(['StudyDate', 'StudyTime','LKW time','LKW date','LKWTime'],axis=1)
Included_Subjects_Aggregate

In [ ]:
Included_Subjects_FirstScanTimes['ScanTime'] = Included_Subjects_FirstScanTimes['StudyDate'].apply(pad) + Included_Subjects_FirstScanTimes['StudyTime'].apply(pad)
Included_Subjects_FirstScanTimes['ScanTime'] = pd.to_datetime(Included_Subjects_FirstScanTimes['ScanTime'],format='%Y%m%d%H%M%S') 
Included_Subjects_FirstScanTimes = Included_Subjects_FirstScanTimes.drop(['StudyDate','StudyTime'], axis=1)
Included_Subjects_FirstScanTimes.columns = ['PatientID', 'FirstScanType', 'FirstScanTime']

In [ ]:
Included_Subjects_FirstScanTimes

In [ ]:
merged_df = Included_Subjects_Aggregate.merge(
    Included_Subjects_FirstScanTimes[['PatientID', 'FirstScanType', 'FirstScanTime']],
    on='PatientID',
    how='left'  # Use 'inner' if you only want to keep rows that have matching PatientID in both dataframes
)
merged_df

In [ ]:
ISA = merged_df[['PatientID', '>4s', '>6s', '>8s', '>10s', '<38%', '<34%', '<30%', '<20%',
                    'CBF30_adj_0cc', 'P:C(30)>=1.8', 'P-C', 'CBF38_adj_0cc', 'P:C(38)>=1.8',
                    'PatientBirthDate', 'Lat', 'FirstScanType','FirstScanTime','ScanTime',
                    'LKWDiff']]
ISA

In [ ]:
arterial_sites = Aggregate[~pd.isna(Aggregate['Primary Arterial Site'])]
arterial_sites = arterial_sites[arterial_sites['PatientID'].isin(ISA['PatientID'].values)].drop_duplicates('PatientID').sort_index()
arterial_sites = arterial_sites[['PatientID','Primary Arterial Site']]
arterial_sites

In [ ]:
ISA = pd.merge(ISA,arterial_sites,left_on='PatientID',right_on='PatientID')
ISA

In [ ]:
ISA['PatientID'] = ISA['PatientID'].apply(str)
ISA['PatientBirthDate'] = ISA['PatientBirthDate'] .apply(int)
ISA

In [ ]:
charlen_total_notes = []
charlen_first_notes = []

for id,scantime in ISA[['PatientID','ScanTime']].values:
    Notes_included = Notes[Notes['PAT_MRN_ID']== str(id)]
    Notes_included['Entry DateTime'] = Notes_included['ENTRY_INST_LOCAL_DTTM'].apply(pd.to_datetime)
    scantime = pd.to_datetime(scantime)
    Notes_included = Notes_included[scantime > Notes_included['Entry DateTime']]
    Notes_included = Notes_included[scantime - Notes_included['Entry DateTime'] < pd.Timedelta('1w')]
    merged_notes = Notes_included.groupby('NOTE_ID')['NOTE_TEXT'].agg(' '.join)
    if merged_notes.shape[0] == 0:
        continue
    Notes_included = pd.merge(Notes_included, merged_notes, on='NOTE_ID',how='left')
    Notes_included.drop('NOTE_TEXT_x',axis=1,inplace=True)
    Notes_included.drop_duplicates('NOTE_TEXT_y',inplace=True)
    Notes_included['line length'] = Notes_included['NOTE_TEXT_y'].apply(len)
    Notes_included = Notes_included.iloc[::-1].reset_index()
    total_Note = ' '.join(Notes_included['NOTE_TEXT_y'].values)
    first_Note = Notes_included['NOTE_TEXT_y'][0]
    charlen_first_notes.append(len(first_Note))
    charlen_total_notes.append(len(total_Note))

In [ ]:
print('dist of total note lengths in week prior to scan')
print(np.percentile(charlen_total_notes,25))
print(np.percentile(charlen_total_notes,50))
print(np.percentile(charlen_total_notes,75))

plt.hist(charlen_total_notes,30)
plt.show()

print('dist of first note lengths in week prior to scan')
print(np.percentile(charlen_first_notes,25))
print(np.percentile(charlen_first_notes,50))
print(np.percentile(charlen_first_notes,75))

plt.hist(charlen_first_notes,30)
plt.show()

In [ ]:
cutoffs = [500,1000,5000]

In [ ]:
for cutoff in cutoffs:
    text = dict()
    for id,scantime in ISA[['PatientID','ScanTime']].values:
        Notes_included = Notes[Notes['PAT_MRN_ID']== str(id)]
        Notes_included['Entry DateTime'] = Notes_included['ENTRY_INST_LOCAL_DTTM'].apply(pd.to_datetime)
        Notes_included = Notes_included[scantime > Notes_included['Entry DateTime']]
        Notes_included = Notes_included[scantime - Notes_included['Entry DateTime'] < pd.Timedelta('1w')]
        merged_notes = Notes_included.groupby('NOTE_ID')['NOTE_TEXT'].agg(' '.join)
        if merged_notes.shape[0] == 0:
            continue
        Notes_included = pd.merge(Notes_included, merged_notes, on='NOTE_ID',how='left')
        Notes_included.drop('NOTE_TEXT_x',axis=1,inplace=True)
        Notes_included.drop_duplicates('NOTE_TEXT_y',inplace=True)
        Notes_included['line length'] = Notes_included['NOTE_TEXT_y'].apply(len)
        Notes_included = Notes_included.iloc[::-1]
        Notes_included['length sum'] = Notes_included['line length'].cumsum()
        Notes_included['met threshold?'] = Notes_included['length sum'] >= cutoff
        if Notes_included['met threshold?'].sum() == 0:
            row_index = Notes_included.shape[0]
        else:
            row_index = Notes_included['met threshold?'].argmax()
        Notes_included = Notes_included.iloc[:row_index+1]
        Note = ' '.join(Notes_included['NOTE_TEXT_y'].values)
        text[id] = Note

    Text = pd.DataFrame(text.values(),index=text.keys(),columns=['text'])
    Text.to_csv('Unprocessed Text (120) (cutoff ' + str(cutoff) + ')')

In [ ]:
ISA.to_csv('Included Aggregate Data (120)')